In [18]:
from traffic_generator import traffic_generator
import threading
import time

In [ ]:
"""
关于数据格式
逻辑拓扑应该呈现为有向环形, 表现为:
TOPOLOGY = ["Node_i", "Node_j", "Node_k", "Node_l", "Node_m"]
每个Node的Segment的数据量依照DICT方式给出, 表现为:
DATA_DICT = {
    "Node_i":[10, 20, 30, 40, 1],
    "Node_j":[15, 40, 25, 35, 1],
    "Node_k":[12, 18, 40, 25, 1],
    "Node_l":[40, 12, 15, 23, 1],
    "Node_m":[20, 13, 25, 11, 12]
}
"""

In [19]:
TOPOLOGY = ["Node_i", "Node_j", "Node_k", "Node_l", "Node_m"]
DATA_DICT = {
    "Node_i":[10, 20, 30, 40, 1],
    "Node_j":[15, 40, 25, 35, 1],
    "Node_k":[12, 18, 40, 25, 1],
    "Node_l":[40, 12, 15, 23, 1],
    "Node_m":[20, 13, 25, 11, 12]
}
link_bandwidth = 2 # 应该以邻接矩阵给出
traffic_distribution = "uniform"
link_delay = 5

In [ ]:
# Ring-AllReduce_algorithms

NODES_NUM = len(TOPOLOGY)
MAX_STEPS = 2 * (NODES_NUM - 1)
FINAL_DICT = DATA_DICT
for step in range(MAX_STEPS):
    REDUCE_FLAG = True
    if step >= (NODES_NUM - 1):
        REDUCE_FLAG = False
    for node_id in range(NODES_NUM):
        data_index = ((node_id + step*(NODES_NUM-1)) % NODES_NUM)
        # print(data_index)
        if node_id == NODES_NUM-1:
            src_node = TOPOLOGY[node_id]
            dst_node = TOPOLOGY[0]
            if REDUCE_FLAG:
                FINAL_DICT[dst_node][data_index] += FINAL_DICT[src_node][data_index]
                continue
            FINAL_DICT[dst_node][data_index] = FINAL_DICT[src_node][data_index]
            continue
        src_node = TOPOLOGY[node_id]
        dst_node = TOPOLOGY[node_id+1]
        if REDUCE_FLAG:
            FINAL_DICT[dst_node][data_index] += FINAL_DICT[src_node][data_index]
            continue
        FINAL_DICT[dst_node][data_index] = FINAL_DICT[src_node][data_index]

print(FINAL_DICT)

{'Node_i': [97, 103, 135, 134, 16], 'Node_j': [97, 103, 135, 134, 16], 'Node_k': [97, 103, 135, 134, 16], 'Node_l': [97, 103, 135, 134, 16], 'Node_m': [97, 103, 135, 134, 16]}


In [12]:
# Broadcast_algorithms (Based on Binary-Tree)

father_nodes_list = ["Node_i"]
son_nodes_list = ["Node_j", "Node_k", "Node_l", "Node_m", "Node_n", "Node_o"]
data_size = 10000

link_bandwidth = 100
link_delay = 20
# 这里的link_delay和link_bd都应该从邻接矩阵里面取而不是直接给出
SUM_TIME = 0
NODES_CNT = 1 # 记录所有收到数据的节点个数, 这里也可以用log2来做因为是基于二叉树的, 但是本质没差. 加一个计数器好像更简单一些
while NODES_CNT < len(father_nodes_list) + len(son_nodes_list):
    temp_father_nodes_list = []
    max_time = 0
    # max_time用来记录每层Tree传输的时候的最大时间, 用这个最大时间作为该层的传输时间, 最终将多层的传输时间求和得到Broadcast的完成时间
    for father_node in father_nodes_list:
        for index, son_node in enumerate(son_nodes_list):
            time = data_size / link_bandwidth + link_delay
            # 正确的方法是这里应该根据不同的link_bd和link_delay计算出不同的两个node之间的时间
            if time > max_time:
                max_time = time
            NODES_CNT += 1
            _ = son_nodes_list.pop(index)
            temp_father_nodes_list.append(son_node)
            break
    SUM_TIME += max_time
    father_nodes_list += temp_father_nodes_list
print(SUM_TIME)

240.0
